In [133]:
import requests
import re
import numpy as np

# COnfig
## Define Protein Sequences to run here:
protein_sequences = [ "Q15836", "Q01524", "P02652", "P02818", "O60516", "P05114", "Q9BXV9", "Q9BV40", "P00974", "P0AFE4"]

In [134]:
# Using Uniprot API to get protein sequence

# Query to fetch 10 random sequences of length 100
"https://rest.uniprot.org/uniprotkb/search?query=(length:[100 TO {100}])&format=fasta&size=10"

def getProtSeq(accession):
    url = f"https://www.uniprot.org/uniprot/{accession}.fasta"
    response = requests.get(url)
    return response.text

# sample
print(getProtSeq("P00974"))
print(getProtSeq("Q9BV40"))

>sp|P00974|BPT1_BOVIN Pancreatic trypsin inhibitor OS=Bos taurus OX=9913 PE=1 SV=2
MKMSRLCLSVALLVLLGTLAASTPGCDTSNQAKAQRPDFCLEPPYTGPCKARIIRYFYNA
KAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGAIGPWENL

>sp|Q9BV40|VAMP8_HUMAN Vesicle-associated membrane protein 8 OS=Homo sapiens OX=9606 GN=VAMP8 PE=1 SV=1
MEEASEGGGNDRVRNLQSEVEGVKNIMTQNVERILARGENLEHLRNKTEDLEATSEHFKT
TSQKVARKFWWKNVKMIVLICVIVFIIILFIVLFATGAFS



In [ ]:
def format_fasta(seq):
    data = {"header": {}, "sequence": ""}
    header = r'^>(sp|tr)\|([A-Z0-9]+)\|([A-Z0-9_]+)\s+(.+?)\s+OS=(.+?)\s+OX=(\d+)(?:\s+GN=([^\s]+))?(?:\s+PE=(\d+))?(?:\s+SV=(\d+))?' # Makes GN an optional field cause for some reason some data didnt have it
    filtered_seq = re.match(header, seq)

    if seq == "":
        return print("EMPTY DATA FETCHED. Check the accession again.")
    
    
    if filtered_seq:
        data["header"]["accession"] = filtered_seq.group(2)
        data["header"]["name"] = filtered_seq.group(3)
        data["header"]["description"] = filtered_seq.group(4)
        data["header"]["organism"] = filtered_seq.group(5)
        data["header"]["ox"] = filtered_seq.group(6)
        data["header"]["gene"] = filtered_seq.group(7)
        data["header"]["pe"] = filtered_seq.group(8)
        data["header"]["sv"] = filtered_seq.group(9)
    
        data["sequence"] = seq.splitlines()[1:]
    return data

# format_fasta(getProtSeq("P69905"))


In [136]:
class protein:
    def __init__(self, accession):
        self.accession = accession
        self.sequence = None
        self.header = None

    def fetch_sequence(self):
        fasta_data = getProtSeq(self.accession)
        formatted_data = format_fasta(fasta_data)
        self.sequence = formatted_data["sequence"]
        self.header = formatted_data["header"]

    def format_sequence(self):
        seq = ''.join(self.sequence)
        seq = seq.replace("\n", "")
        return seq


    def __str__(self):
        return f"Protein {self.accession} - {self.header['description']}"
    
    def getNmer(self,n):
        nmer = []
        seq = self.format_sequence()
        for i in range(len(seq) - n + 1):
            nmer.append(seq[i:i+n])
        return nmer
    
    def findMotif(self):
        seq = self.format_sequence()

        motif = input("Enter the regex motif to search for: ")

        matches = re.finditer(motif, seq) # multiple times
        for match in matches:
            start = match.start()
            end = match.end()
            print(f"Match found at position {start} to {end}: {match.group()}")

        return 


In [137]:
protein_objects = [] # This are class objects
for accession in protein_sequences:
    prot = protein(accession)
    prot.fetch_sequence()
    protein_objects.append(prot)

In [138]:

#Example Uasage
print(protein_objects[0].header)
print(protein_objects[0].sequence)


{'accession': 'Q15836', 'name': 'VAMP3_HUMAN', 'description': 'Vesicle-associated membrane protein 3', 'organism': 'Homo sapiens', 'ox': '9606', 'gene': 'VAMP3', 'pe': '1', 'sv': '3'}
['MSTGPTAATGSNRRLQQTQNQVDEVVDIMRVNVDKVLERDQKLSELDDRADALQAGASQF', 'ETSAAKLKRKYWWKNCKMWAIGITVLVIFIIIIIVWVVSS']


### Get nMer of the Sequence (Define it below)

In [139]:

nMer_length = 3

for prot in protein_objects:
    print(prot)
    print(prot.getNmer(nMer_length))
    print("\n")

Protein Q15836 - Vesicle-associated membrane protein 3
['MST', 'STG', 'TGP', 'GPT', 'PTA', 'TAA', 'AAT', 'ATG', 'TGS', 'GSN', 'SNR', 'NRR', 'RRL', 'RLQ', 'LQQ', 'QQT', 'QTQ', 'TQN', 'QNQ', 'NQV', 'QVD', 'VDE', 'DEV', 'EVV', 'VVD', 'VDI', 'DIM', 'IMR', 'MRV', 'RVN', 'VNV', 'NVD', 'VDK', 'DKV', 'KVL', 'VLE', 'LER', 'ERD', 'RDQ', 'DQK', 'QKL', 'KLS', 'LSE', 'SEL', 'ELD', 'LDD', 'DDR', 'DRA', 'RAD', 'ADA', 'DAL', 'ALQ', 'LQA', 'QAG', 'AGA', 'GAS', 'ASQ', 'SQF', 'QFE', 'FET', 'ETS', 'TSA', 'SAA', 'AAK', 'AKL', 'KLK', 'LKR', 'KRK', 'RKY', 'KYW', 'YWW', 'WWK', 'WKN', 'KNC', 'NCK', 'CKM', 'KMW', 'MWA', 'WAI', 'AIG', 'IGI', 'GIT', 'ITV', 'TVL', 'VLV', 'LVI', 'VIF', 'IFI', 'FII', 'III', 'III', 'III', 'IIV', 'IVW', 'VWV', 'WVV', 'VVS', 'VSS']


Protein Q01524 - Defensin-6
['MRT', 'RTL', 'TLT', 'LTI', 'TIL', 'ILT', 'LTA', 'TAV', 'AVL', 'VLL', 'LLV', 'LVA', 'VAL', 'ALQ', 'LQA', 'QAK', 'AKA', 'KAE', 'AEP', 'EPL', 'PLQ', 'LQA', 'QAE', 'AED', 'EDD', 'DDP', 'DPL', 'PLQ', 'LQA', 'QAK', 'AKA', 'KAY', 'AY

In [140]:
# Example usage of finding MOTIF:
prot_seq = protein_objects[0]
prot_seq.findMotif() # Input: P.{3}


Match found at position 4 to 5: P


### Getting Shannon Entropy

In [172]:
# REFERENCE: http://imed.med.ucm.es/Tools/svs_help.html
"""
= 0.0 : Fully conserved (only 1 amino acid appears)
< 1.0 : Highly conserved position
< 2.0 : Conserved
> 2.0 : Variable position
--Max : All amino acids equally likely
"""

def shannonEntropyFunction(prob):
    val = 0
    for p in prob:
        if p > 0:  # avoid log(0) errors
            val += -p*np.log2(p)
    return val


sequence_list = np.array([p.format_sequence().strip() for p in protein_objects])

def shannonEntropy(seq_list):
    no_of_seq = len(seq_list)

    length_of_seq = len(seq_list[0])

    # Check for Length
    for i in seq_list:
        if len(i) != length_of_seq:
            print(length_of_seq)
            print(len(i))
            print("Sequences are of not the same length. Exiting...")
            return None
        
    Probabilities = []
    for j in range(length_of_seq):
        amino_acids = [seq[j] for seq in seq_list]
        # print(amino_acids)

        amino_feq =  {}

        for amino in amino_acids:
            if amino in amino_feq:
                amino_feq[amino] += 1
            if amino not in amino_feq:
                amino_feq[amino] = 1

        amino_dict = {aa: count / no_of_seq for aa, count in amino_feq.items()}
        amino_feq = list(amino_dict.values())
        # print(amino_dict)

        Prob = shannonEntropyFunction(amino_feq)
        Probabilities.append(Prob)
    return Probabilities

Probs = np.array(shannonEntropy(sequence_list))
print(Probs)
print(np.unique(Probs))


[0.         2.52192809 2.64643934 1.96096405 2.84643934 2.92192809
 2.72192809 2.72192809 2.52192809 2.64643934 2.44643934 2.44643934
 2.52192809 2.92192809 2.32192809 2.32192809 2.72192809 2.72192809
 2.92192809 3.32192809 2.92192809 2.92192809 2.52192809 2.64643934
 2.84643934 3.12192809 2.64643934 2.44643934 2.92192809 2.72192809
 2.92192809 3.12192809 3.12192809 2.64643934 2.72192809 3.12192809
 2.92192809 3.12192809 2.64643934 3.12192809 2.72192809 2.72192809
 3.12192809 3.12192809 3.32192809 3.12192809 2.64643934 3.12192809
 3.12192809 3.12192809 2.92192809 2.92192809 2.72192809 2.64643934
 2.72192809 2.92192809 2.92192809 2.92192809 2.72192809 3.12192809
 2.72192809 2.92192809 2.44643934 2.64643934 2.72192809 2.92192809
 2.92192809 2.92192809 3.12192809 3.32192809 3.32192809 2.84643934
 3.12192809 2.92192809 3.12192809 3.12192809 2.64643934 3.12192809
 2.92192809 3.12192809 2.64643934 2.64643934 2.64643934 2.92192809
 2.72192809 3.12192809 3.12192809 3.12192809 2.92192809 2.9219